In [3]:
import pandas as pd
import ast
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(context="talk", style="whitegrid", palette="colorblind")

In [4]:
asserts = pd.read_csv(
    "data/shome2023notebook/assert-content.csv",
    index_col=0,
)
asserts

,notebook,assert,source
12,data/assert_notebooks/realliyifei/ml-project-h...,assert sum(np.array(field_dims)) == X_train.sh...,"field_dims = [1, 4, 2, 3, 1, 1, 2, 2, 1, 1, 1,..."
24,data/assert_notebooks/realliyifei/ml-project-h...,assert data.shape[1] == len(sparse_features) +...,"sparse_features = data.loc[:, data.dtypes==np...."
12,data/assert_notebooks/declanvk/data301-project...,assert raw_data.pclass.isnull().value_counts()...,assert raw_data.pclass.isnull().value_counts()...
12,data/assert_notebooks/declanvk/data301-project...,"assert list(raw_data.sex.unique()) == ['male',...",assert raw_data.pclass.isnull().value_counts()...
12,data/assert_notebooks/declanvk/data301-project...,assert raw_data.age.isnull().value_counts()[Fa...,assert raw_data.pclass.isnull().value_counts()...
...,...,...,...
14,data/quaranta2021kgtorrent/KT_dataset/epocxy_m...,assert df_train['cp_time_dose'].nunique() == d...,assert df_train['cp_time_dose'].nunique() == d...
9,data/quaranta2021kgtorrent/KT_dataset/tunguz_h...,assert X_test.shape[0] == id_test.shape[0] // 3,# Release RAM of the training data \nif 'v_raw...
27,data/quaranta2021kgtorrent/KT_dataset/samusram...,assert object_id is not None or object_df is n...,"def plot_agg_light_curve(object_id=None, objec..."
6,data/quaranta2021kgtorrent/KT_dataset/motoight...,"assert num_layers == len(num_filters), 'ERROR:...",import torch\nimport torch.nn as nn\nimport to...


# Identify interesting assertions

We need to identify interesting asserts which we want to present as in-depth case-studies.

Identifying something interesting from a random sample is all about luck. In this section I am exploring a few unsupervised techniques to surface interesting assertions we can analyse in more detail:

+ bi-grams and tri-grams
+ clustering
+ topic-modelling

## mono-grams

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    stop_words=["assert"],
)
X = vectorizer.fit_transform(asserts["assert"])
occurances = pd.DataFrame(
    data={"freq": X.toarray().sum(axis=0)},
    index=vectorizer.get_feature_names_out(),
)
occurances = occurances.sort_values(by=["freq"], ascending=False)

In [7]:
occurances.describe()

,freq
count,22944.000000
mean,2.205275
std,17.780126
min,0.022746
25%,0.511930
50%,0.705716
75%,1.206568
max,1520.924514


In [16]:
occurances.loc[occurances.index.str.contains("x")]

,freq
x_train,139.577913
max,109.106557
index,104.059178
x_test,100.837775
exists,98.390991
...,...
xenium,0.187737
_gexpr,0.183458
appendix,0.175410
1dqfywetx614r49kuve3cbzwvo6qhvrvh,0.174554


In [20]:
def get_freq_sum(tokens: list[str]) -> float:
    return sum(
        map(
            lambda token: occurances.loc[token, "freq"],
            tokens,
        )
    )


asserts.loc[:, "tokens"] = asserts.loc[:, "assert"].apply(vectorizer.build_analyzer())
asserts.loc[:, "sum_freq"] = asserts.loc[:, "tokens"].apply(get_freq_sum)
asserts.loc[:, "bin"] = pd.qcut(asserts.loc[:, "sum_freq"], 10, labels=False)
asserts["bin"].value_counts()

bin
7    2677
1    2556
9    2555
0    2555
4    2555
5    2555
2    2554
6    2554
3    2554
8    2433
Name: count, dtype: int64

In [23]:
for idx in asserts["bin"].unique():
    asserts.loc[asserts["bin"] == idx].sample(100, random_state=42).to_csv(f"data/shome2023notebook/decile-{idx}.csv")